Para demonstrar a utilização do python como uma ferramenta de resolução de problemas de engenharia de petróleo, criei este notebook que mostra passo a passo como resolver calcular o influxo acumulado de água em um reservatório de petroleo.

Utilizei a questão 6.1, pagina: 359, do livro Engenharia de Reservatórios de Petróleo dos autores Adalberto José Rosa, Renato de Souza Carvalho e José Augusto Daniel Xavier

# Questão 6.1: O histórico das pressões no contato óleo/água original de um reservatório é apresentado na tabela a seguir:

| Tempo (ano) | Pressão (kgf/cm²) |
| --- | --- |
| 0 | 220 |
| 1 | 180 |
| 2 | 160 |
| 3 | 160 |
| 4 | 160 |

Outros dados diponíveis são:

| Variáveis | Símbolo | Valor |
| --- | --- | --- |
| Raio do reservatório | *r<sub>o<sub>* | 1200 m |
| Espessura da formação | *h* | 10 m |
| Porosidade | Φ | 0,15 |
| Permeabilidade | *k* | 50 md |
| Viscosidade da água | μ | 1,0 cp |
| Compressilidade da Formação | *c<sub>f<sub>* | 55x10<sup>-6</sup> |
| Compressibilidade da água | *c<sub>w<sub>* | 45x10<sup>-6<sup> |

Admitindo que durante os primeiros 4 anos de produção o sitema real possa ser representado por um reservatório associado a um aquífero radial infinito, determe o influxo acumulado de água:

## (a) Ao final do 1° ano de produção.

### 1° Passo: Importe as bibliotecas necessárias para a resolução do problema.

In [134]:
import pandas as pd #Manipulação de dados
import numpy as np #Calculos cientificos
import plotly.express as px #Plotar gráficos
from sklearn.linear_model import LinearRegression #Regressão linear
from sklearn.model_selection import train_test_split #Dividir dados para treinamento e teste

### 2° Passo:Transforme os dados da questão em variáveis

In [249]:
ro = 1200 #Raio do reservatório
h = 10 #Espessura da formação
Φ = 0.15 #Porosidade
k = 50 #Permeabilidade
μ = 1.0 #Viscosidade da água
cf = 55*10**(-6) #Compressilidade da Formação
cw = 45*10**(-6) #Compressilidade da Formação
π = np.pi #pi
f = (2*π)/(2*π) #angulo de invasão

### 3° Passo: Transforme o histórico de pressão em DataFrame

In [243]:
hist_pressao_df = pd.DataFrame(data = [[0,220],[1,180],[2,160],[3,160],[4,160]], columns = ['Tempo [ano]', 'Pressão [kgf/cm²]']) #[t=0,P=220],[t=1,P=180],[t=2,P=160],[t=3,P=160],[t=4,P=160]

hist_pressao_df #apresenta o DataFrame para checagem

,Tempo [ano],Pressão [kgf/cm²]
0,0,220
1,1,180
2,2,160
3,3,160
4,4,160


Você utilizará este DataFrame até o fim do processo

### 4° Passo: Calcule a compressibilidade total (ct) e a constante de influxo de água do aquífero (U)

In [251]:
ct = cw + cf # Calcula a compressibilidade total
U = 2*π*f*Φ*ct*h*ro**2 #Calcula a constante de influxo do aquífero

### 5° Passo: Crie uma função para o calculo do tempo admensional

In [252]:
def tempo_adimensional(t): #Cria a função para o calculo do tempo admensional
    tempo_admensional = (0.0003484*24*365*k)*t/(Φ*μ*ct*ro**2)
    return tempo_admensional

### 6° Passo: Importe os dados de Influxo admensional vs tempo admensional, plote os dados com um gráfico de dispersão

In [253]:
tabela_aquifero_infinito = pd.read_csv("infinity aquifer.csv", sep = ";") #importa dos dados de td vs wd

In [276]:
fig = px.scatter(tabela_aquifero_infinito, x = "td", y = "wd") #cria grafico td vs wd
fig.show() #apresenta gráfico de td vs wd

### 7° Passo: Analise a curva e identifique qual o melhor modelo de regressão pode ser usado e aplique-o.
<br><br>Pela distribuição dos pontos no gráfico, afirma-se que um modelo de regressão linear simples se adequa ao cenário.

In [255]:
x = tabela_aquifero_infinito.iloc[0:100, :-1].values #vincula a variavel x aos dados de td
y = tabela_aquifero_infinito.iloc[0:100, 1].values #vincula a variavel y aos dados de wd
xTrain, xTest, yTrain, yTest = train_test_split(x, y, test_size = 1/3, random_state = 0) #Separa a amostra em dados de treino e dados de teste.
linearRegressor = LinearRegression() #Cria uma variável para armazenar a regressão
linearRegressor.fit(xTrain, yTrain) #Ajusta a regressão com as variáveis de treinamento
print ("O R² da regressão linear utilizada", linearRegressor.score(xTrain,yTrain)) #Apresenta o R², que indica a representatividade da curva em relação aos dados

O R² da regressão linear utilizada 0.9911517563632642


Como o valor do R² é de 99%, utilizaremos os coeficientes do modelo para prever os valores de wd, dispensando a consulta de tabela e reduzindo o tempo de calculo.

In [256]:
def wd(td): #cria a função de previsão de wd
    if td <= 0: #Para valores de td <= 0 o wd será 0
        wd = 0
    else: # Para valores de td > 0 o wd pode ser previsto por uma equação do primeiro grau y = ax + b
        wd = float(linearRegressor.coef_)*td + float(linearRegressor.intercept_) # Analogamente a função do primeiro grau Y = ax + b, a = linearRegressor.coef_ e b = linearRegressor.intercept_
    return wd

### 8° Passo: Calcule a queda de pressão ao final de 1 ano de produção

In [257]:
ΔPi = (hist_pressao_df.iloc[0,1] - hist_pressao_df.iloc[1,1])/2 #Calcula o ΔP para 1 ano de produção

In [258]:
wedi = U * ΔPi * wd(7.0648) #Calcula o influxo acumulado ao final do 1° ano de produção
print ("O influxo acumulado de água ao final do 1° ano de produção é", wedi, "m³.")

O influxo acumulado de água ao final do 1° ano de produção é 153076.4678503576 m³.


## (b) Ao final do 4° ano de produção.

### 1° Passo: Cácule os influxos admensionais e as variações de pressão para cada intervalo e adicioneos ao DataFrame hist_pressao_df.

In [259]:

hist_pressao_df['Influxo admensional'] = [wd(tempo_adimensional(hist_pressao_df.iloc[4,0]-hist_pressao_df.iloc[0,0])),# wd(td=(4 anos - 0 ano))
                                          wd(tempo_adimensional(hist_pressao_df.iloc[4,0]-hist_pressao_df.iloc[1,0])),# wd(td=(4 anos - 1 ano))
                                          wd(tempo_adimensional(hist_pressao_df.iloc[4,0]-hist_pressao_df.iloc[2,0])),# wd(td=(4 anos - 2 anos))
                                          wd(tempo_adimensional(hist_pressao_df.iloc[4,0]-hist_pressao_df.iloc[3,0])),# wd(td=(4 anos - 3 anos))
                                          wd(tempo_adimensional(hist_pressao_df.iloc[4,0]-hist_pressao_df.iloc[4,0]))]# wd(td=(4 anos - 4 anos))

hist_pressao_df['ΔPj'] = [(hist_pressao_df.iloc[0,1]-hist_pressao_df.iloc[1,1])/2, #(ΔP0-ΔP1)/2
                          (hist_pressao_df.iloc[0,1]-hist_pressao_df.iloc[2,1])/2, #(ΔP0-ΔP2)/2
                          (hist_pressao_df.iloc[1,1]-hist_pressao_df.iloc[3,1])/2, #(ΔP1-ΔP2)/2
                          (hist_pressao_df.iloc[2,1]-hist_pressao_df.iloc[4,1])/2, #(ΔP2-ΔP3)/2
                          (hist_pressao_df.iloc[4,1]-hist_pressao_df.iloc[4,1])/2] #(ΔP3-ΔP3)/2

O Data Frame hist_pressao_df estará neste formato

In [260]:
hist_pressao_df# Apresenta o DataFrame

,Tempo [ano],Pressão [kgf/cm²],Influxo admensional,ΔPj
0,0,220,14.834981,20.0
1,1,180,11.769836,30.0
2,2,160,8.704690,10.0
3,3,160,5.639545,0.0
4,4,160,0.000000,0.0


### 2° Passo: Multiplique a coluna "Influxo admensional" pela coluna "ΔPj" e adcione a coluna resultado ao seu DataFrame.

In [261]:
hist_pressao_df['ΔPj*Wd'] = hist_pressao_df['Influxo admensional']*hist_pressao_df['ΔPj'] #Cria a coluna "ΔPj*Wd"

In [262]:
hist_pressao_df #Apresenta o Data Frame

,Tempo [ano],Pressão [kgf/cm²],Influxo admensional,ΔPj,ΔPj*Wd
0,0,220,14.834981,20.0,296.699624
1,1,180,11.769836,30.0,353.095075
2,2,160,8.704690,10.0,87.046905
3,3,160,5.639545,0.0,0.000000
4,4,160,0.000000,0.0,0.000000


### 3° Passo: Some os valores da coluna "ΔPj*Wd" e multiple pela constante de influxo do aquífero para obter o Influxo acumulado ao final do 4° ano de produção.

In [264]:
We = U*hist_pressao_df['ΔPj*Wd'].sum()#Calcula o influxo acumulado
print ("O influxo acumulado de água ao final do 4° ano de produção é", We, "m³.")

O influxo acumulado de água ao final do 4° ano de produção é 1000017.8660260225 m³.


## Análise dos resultados

In [273]:
resultados_df = pd.DataFrame(data = [[156892,wedi],[1087117,We]], columns = ['Resultado Esperado', 'Resultado Calculado'], index =['Wed 1° ano','Wed 4° ano']) #Cria DataFrame com os resultados esperado e calculados

In [274]:
resultados_df['Erro Absoluto'] = (resultados_df['Resultado Esperado']-resultados_df['Resultado Calculado'])/(resultados_df['Resultado Esperado']) #Cria a coluna "Erro Absoluto"
resultados_df #Apresenta DataFrame Final

,Resultado Esperado,Resultado Calculado,Erro Absoluto
Wed 1° ano,156892,1.530765e+05,0.024319
Wed 4° ano,1087117,1.000018e+06,0.080119


Os erros absolutos mostram que o calculo do influxo acumulado  através do metodo apresenta resultados proximos aos valores esperados.